In [1]:
import pandas as pd
import seaborn as sn
from pyspark import SparkContext, SparkConf

In [2]:
conf = SparkConf().setAppName('project')
sc = SparkContext(conf=conf)
sc

<SparkContext master=local[*] appName=project>

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('project').config('spark.some.config.option', "some-value").getOrCreate()

In [4]:
icd10 = ['I501', 'I5020', 'I5021', 'I5022', 'I5023', 'I5030', 'I5031', 'I5032', 'I5033', 'I5040', 'I5041', 'I5042', 'I5043', 'I509',  'I50810','I50811', 'I50812', 'I50813', 'I50814', 'I5082', 'I5083', 'I5084', 'I5089',  'I42', 'I420', 'I423', 'I424', 'I425', 'I426', 'I427', 'I428', 'I429', '143', '1430', '1431', '1432', '1438',  'I110']
icd09 = ['4280', '4281', '42820', '42821', '42822', '42823', '42830', '42831' ,'42832', '42833', '42840', '42841' ,'42842', '42843','4289']
icd = icd09+icd10

In [6]:
path = r'mimic-iv-1.0\hosp\diagnoses_icd.csv'
#df = pd.read_csv(path)
df_icd = spark.read.csv(path, header=True)

In [7]:
df_icd2 = df_icd.filter(df_icd.icd_code.isin(icd))

In [8]:
df_subject_ids = df_icd2.select('subject_id').distinct()

In [9]:
subject_ids = df_subject_ids.select("subject_id").toPandas()
subject_ids = subject_ids.subject_id.tolist()

## csvs that have icd code in hosp:
d_icd_diagnoses, d_icd_procedures, diagnoses_icd, procedures_icd

## csvs that hade subject_id in hosp:
services, procedures_icd, prescriptions, poe_detail, poe, pharmacy, microbiologyevents,labevents hcpcsevents, emar_detail, emar, 
## csvs that have neither in hosp:
d_hcpcs, d_labitems

## all csvs in core have subject_id

## all in icu has subject_id except d_items.csv

In [15]:
icd_csv_hosp = ['d_icd_diagnoses', 'd_icd_procedures', 'diagnoses_icd', 'procedures_icd']
sub_id_hosp = ['services', 'prescriptions','labevents', 'poe_detail', 'poe', 'pharmacy', 'microbiologyevents', 'hcpcsevents', 'emar_detail', 'emar']
sub_id_core = ['admissions', 'patients', 'transfers']
sub_id_icu = ['chartevents', 'datetimeevents', 'icustays', 'inputevents', 'outputevents', 'procedureevents']

In [14]:
for csv in icd_csv_hosp:
    path = "mimic-iv-1.0/hosp/"+csv
    #df = pd.read_csv(path)
    df = spark.read.csv(path+".csv", header=True)
    df = df.filter(df.icd_code.isin(icd))
    df.write.option('header', True).csv(path+"_filtered.csv")

In [ ]:
for csv in sub_id_hosp:
    path = "mimic-iv-1.0/hosp/"+csv
    #df = pd.read_csv(path)
    df = spark.read.csv(path+".csv", header=True)
    df = df.filter(df.subject_id.isin(subject_ids))
    df.write.option('header', True).csv(path+"_filtered.csv")

In [ ]:
for csv in sub_id_core:
    path = "mimic-iv-1.0/core/"+csv
    #df = pd.read_csv(path)
    df = spark.read.csv(path+".csv", header=True)
    df = df.filter(df.subject_id.isin(subject_ids))
    df.write.option('header', True).csv(path+"_filtered.csv")

In [ ]:
for csv in sub_id_icu:
    path = "mimic-iv-1.0/icu/"+csv
    #df = pd.read_csv(path)
    df = spark.read.csv(path+".csv", header=True)
    df = df.filter(df.subject_id.isin(subject_ids))
    df.write.option('header', True).csv(path+"_filtered.csv")